In [5]:
from pylab import *
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from textwrap import wrap
import random

###############################################################################
# SOUS PROGRAMMES ET FONCTIONS
###############################################################################
# calcul de la valeur de la derivee du potentiel en un point x
def derivpot(z, Potentiel):
        h=0.00001
        pot=(Potentiel(z+h)-Potentiel(z))/h
        return pot
###############################################################################
###############################################################################
# creation de la fonction resoudre qui donne position et vitesses par la methode leapfrog (coeur du programme)
def branche(xb,vb, Potentiel, Mm, index1):
########################
# Integration "leapfrog"
    index2=0
    i=1
    k0=derivpot(xb[i],Potentiel)/m
    vb[i + 1] = vb[i] - dtp*(k0-((b*v0)**2)/xb[i]**3)
    xb[i + 1] = xb[i] + dtp*vb[i + 1]
#    print "2", xb[i + 1]
    while xb[i]<80:
        k0=derivpot(xb[i],Potentiel)/m
        vb[i + 1] = vb[i] - dtp*(k0-((b*v0)**2)/xb[i]**3)
        xb[i + 1] = xb[i] + dtp*vb[i + 1]
#        print xb[i + 1]

        index2, egap = Landau(xb[i-1],xb[i],xb[i+1],vb[i],vb[i+1],index1,Potentiel)
        if not(index1==index2):
#            print 'hop from', index1, 'to',index2, 'at R=', xb[i]
            k0=derivpot(xb[i],Potentiel)/m
            vb[i + 1] = vb[i] - dtp*(k0-((b*v0)**2)/xb[i]**3)
            xb[i + 1] = xb[i] + dtp*vb[i + 1]
            i+=1

            break
        i+=1
#        print i, mod(i,10)

##       if abs(xb[i+1])<1.1:
##           i=i+1
##           continue
##
##       else:
##
##           index2, L_Z, vb[l], vb[l+1] =Landau(xb[i-1],xb[i],xb[i+1],vb[i],vb[i+1],index1, Potentiel)
##           i=i+1
##
##           if index2==index1:continue
##
##           else:
##
##
##               break
##
##
    flg=1
    if  xb[i]>80:
         flg=0
    return xb,vb,index2,index1,i,flg

###############################################################################
# croisement
def Landau(x_1,x_2,x_3,v_2,v_3,asterix,Potentiel1):
        egap=0.0
        dV=zeros(3)
        La_Ze=0
        indef=asterix

        for jpf in range(len(Vg)):

            if not(jpf==asterix):
                Potentiel2=Vg[jpf]
                dV[0]=Potentiel1(x_1)-Potentiel2(x_1)
                dV[1]=Potentiel1(x_2)-Potentiel2(x_2)
                dV[2]=Potentiel1(x_3)-Potentiel2(x_3)
                delta=(dV[1]-dV[0])*(dV[2]-dV[1])

                if delta<0: 
                    d2V=abs((dV[2]-2*dV[1]+dV[0])/dtp**2)
                    La_Ze=exp(-0.5*pi*sqrt(abs(dV[2])**3/d2V))
                    P=random.random()

                    if La_Ze>P and 0.5*m*v_2**2>dV[2]:
#                        print x_1,x_2,x_3
#                        print dV[1]-dV[0],dV[2]-dV[1]
#                        print La_Ze
                        indef=jpf
                        egap=dV[2]
                        break

        return indef, egap
###############################################################################
###############################################################################

# ouverture des fichiers et lecture des donnees
fichierg = open("pec_g.txt", "r")
echelle=1/0.529177
data_g = np.loadtxt(fichierg)
xg_read = data_g[:,0]*echelle

#########################################################################
# creation des fonctions Vgi(x) et Vui(x) qui sont inexees et interpolent les potentiels Vgi et Vui
#typ='cubic'
typ='linear'
g1=interp1d(xg_read, data_g[:,1], kind=typ)
g2=interp1d(xg_read, data_g[:,2], kind=typ)
g3=interp1d(xg_read, data_g[:,3], kind=typ)
g4=interp1d(xg_read, data_g[:,4], kind=typ)
g5=interp1d(xg_read, data_g[:,5], kind=typ)
g6=interp1d(xg_read, data_g[:,6], kind=typ)
g7=interp1d(xg_read, data_g[:,7], kind=typ)
g8=interp1d(xg_read, data_g[:,8], kind=typ)
g9=interp1d(xg_read, data_g[:,9], kind=typ)
Vg = [g1,g2,g3,g4,g5,g6,g7,g8,g9]

#############################################################################
# impressions et plot
plot=1
plot=0
if plot==1:
 figure(figsize=(14,8), dpi=80)
 rcParams['axes.labelsize'] = 20
# Lignes d'unites principales
 plt.grid(b=True, which='major', color='#666666', linestyle='-')
# Lignes de subdivision
 plt.minorticks_on()
 plt.grid(b=True, which='minor', color='#999999', linestyle='-', alpha=0.2)
 xlabel("distance (u.a.)")
 ylabel("potentiel (u.a.)")
 plt.xlim(-0.1,97)
 plt.ylim(-1.2,-0.2)
 for ip in range(1,10):
     plt.plot(xg_read, data_g[:,ip],linewidth=2.0, label="g%d"%(ip,))
    
 plt.title ("courbes de potentiel g", fontsize=22)
 plt.legend(bbox_to_anchor=(0., 1.06, 1., .106), loc='lower left',
           ncol=3, mode="expand", borderaxespad=0.)
 plt.show()


###############################################################################
###############################################################################
# PROGRAMME PRINCIPAL
###############################################################################
# Initialisations des variables
###############################################################################


x0=80.
v0=-0.027  #valeur test a ~10 eV
b=5.

###############################################################################
# masse a prendre en compte et index de la fonction de potentiel
m=918.
potentiel_depart=8

###############################################################################
# entree des parametres temporels - limite de temps tps pas temporel Mm
Mm=2000
tps=10000
tp=np.linspace(0,tps,Mm)
dtp = (tp[1] - tp[0])



Nbmax=10 # nombre de trajets A-R accomplis sur un potentiel de depart donne
gfmax=200 # nombre de branches maximum sur un aller-retour

impactmax= 2  #nombre de paramètres d'impact à considérer

Ntot=zeros((Nbmax)) # vecteur des chemins
Probabl=zeros(9) # vecteur des resultats
Nbdiff=0




b=1.


Nbimpact=0

Probabl=zeros((9,impactmax)) # initialisation de la matrice des résultats


#########################################################################
# boucle principale
######################################################################

while Nbimpact<impactmax:

    Nbdiff=0
    
#########################################################################
# boucle sur les trajets A-R
######################################################################
    while Nbdiff<Nbmax:

        
        ########################
    # Initialisation a 0 des variables, et Conditions initiales
        xinit = zeros(Mm)
        vinit = zeros(Mm)

        indice1 = potentiel_depart-1 # indice du potentiel de depart attention 2 est l'indice Vg de g3 d'ou la soustraction
        Potentiel=Vg[indice1]

        xinit[0] = x0
        vinit[0] = v0
        k0= derivpot(x0,Potentiel)/m
        vinit[1] = vinit[0] - dtp*(k0-((b*v0)**2)/xinit[0]**3)
        xinit[1] = xinit[0] + dtp*vinit[1]

    #    print 
    #    print 
    #    print  Nbdiff
        gf=0
    ##########################################################################
    # boucle sur les branches
    ######################################################################   
        while gf<gfmax:
    #        print gf
            x,v,indice2,indice1,nb,arrt = branche(xinit,vinit,Vg[indice1], Mm, indice1) #si nb=0 fin de la session
            if arrt==0: 
                #print('j ai fini le trajet en', gf,' coups. potentiel de depart g', potentiel_depart, 'potentiel d arrivee g', indice2+1)
                Ntot[Nbdiff]=indice2+1
                Probabl[indice2,Nbimpact]=Probabl[indice2,Nbimpact]+1
                break
            else:
                Potentiel=Vg[indice2]
                xinit = zeros(Mm)
                vinit = zeros(Mm)
                xinit[0] = x[nb]
                vinit[0] = v[nb]

                k0= derivpot(xinit[0],Potentiel)/m
                vinit[1] = vinit[0] - dtp*(k0-((b*v0)**2)/xinit[0]**3)
                xinit[1] = xinit[0] + dtp*vinit[1]
    #            print "1",xinit[1]
                #print('vitesse',vinit[1])
                #print('nouveau potentiel g', indice2+1)
                #graphique(indice1+1, indice2+1, x0, v0, tp, x, v,gf)
                indice1=indice2
                gf=gf+1
        if gf==gfmax:
            print('trop d A-R au pas', Nbdiff)
            break

        Nbdiff=Nbdiff+1
 
    
    
    

    Probabl=Probabl/Nbmax
    
    
    b=b+5/impactmax
    Nbimpact=Nbimpact+1    
    
print(Probabl)    
    
    

###############################################################################
#IL FAUT PREVOIR UNE MATRICE DES RESULTATS Probabl(9, impactmax)
###############################################################################






###############################################################################


[[0.   0.  ]
 [0.   0.1 ]
 [0.   0.1 ]
 [0.01 0.4 ]
 [0.01 0.1 ]
 [0.02 0.1 ]
 [0.   0.  ]
 [0.05 0.1 ]
 [0.01 0.1 ]]
